In [ ]:
!pip install azure-iot-device
!pip install azure-iot-hub
!pip install python-dotenv

In [ ]:
import sys
from dotenv import get_key, find_dotenv
sys.path.append('../utilities/video-analysis/shared/graph_manager')
from graph_manager import GraphManager

# find the environment file .env
env_path = find_dotenv(raise_error_if_not_found=True)

iotHubConnString = get_key(env_path, "IOT_HUB_CONN_STRING")
iotDeviceId = get_key(env_path, "IOT_DEVICE_ID")

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the "deployment.lvaedge_rocketserver_rtspsim.template.json" file
operationsApiVersion = "1.0"  # Must be same as the version number in the "topology.json" file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

### List existing topologies

In [ ]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

In [ ]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

In [ ]:
topologyFile = os.path.abspath("../modules/azureeyeh264module/lva-deployment/cvr-topology.json")
operationParams = {"topologyFile": topologyFile}

In [ ]:
response = graphManager.GraphTopologySet(operationParams)
print(response)

### Set actual topology parameters

In [ ]:
rtspUrl = get_key(env_path, "CROSSING_VIDEO_URL")

#inferencingUrl = get_key(env_path, "PEOPLE_DETECTOR")
inferencingUrl=get_key(env_path, "OPENVINO_DETECTOR")

topologyInstanceName =  "Sample-Graph-1"
topologyName = "InferencingWithHttpExtension"

mediaGraphTopologyParameters = {
          "name": topologyInstanceName,
          "properties": {
            "topologyName": topologyName,
            "description": "Sample graph description",
            "parameters": [
              {
                "name": "rtspUrl",
                "value": rtspUrl
              },
              {
                "name": "rtspUserName",
                "value": "username"
              },
              {
                "name": "rtspPassword",
                "value": "password"
              },
              {
                "name": "inferencingUrl",
                "value": inferencingUrl
              },
              {
                "name": "inferencingUserName",
                "value": "username"
              },
              {
                "name": "inferencingPassword",
                "value": "password"
              },
              {
                "name": "imageEncoding",
                "value": "jpeg"
              },
              {
                "name": "imageScaleMode",
                "value": "pad"
              },
              {
                "name": "frameWidth",
                "value": "1920"
              },
              {
                "name": "frameHeight",
                "value": "1072"
              },
              {
                "name": "frameRate",
                "value": "30"
              }
            ]
          }
        }

#### Set topology instance

In [ ]:
# Set topology instance
response = graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)

### Activate Graph

In [ ]:
# Activate topology instance
operationParams = {"name": topologyInstanceName}

response = graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

### Deactivate Graph

In [ ]:
graphManager.GenericCall("GraphInstanceDeactivate", operationParams)
print(response)



### Delete Graph

In [ ]:
graphManager.GenericCall("GraphInstanceDelete", operationParams)
print(response)

### Delete Topology

In [ ]:
deleteParams = {"name": topologyName}

graphManager.GenericCall("GraphTopologyDelete", deleteParams)
print(response)